# Basic usage

In [1]:
from tmconfpy import Parser

In [2]:
# parse the imap.tmconf and pop3.tmconf files
imap_tmconf = Parser('imap.tmconf', is_filepath=True)
pop3_tmconf = Parser('pop3.tmconf', is_filepath=True)

# display the parsed data as a dictionary
display(
    imap_tmconf.dict,
    pop3_tmconf.dict
)

{'ltm profile imap imap': {'activation-mode': 'require'}}

{'ltm profile pop3 pop3': {'activation-mode': 'require'}}

# Advanced examples

In [3]:
# load test.tmconf
tmconf = Parser('test.tmconf', is_filepath=True)

2024-07-23T17:46:13Z - WARNING - tmconfpy.parser - UNRECOGNIZED LINE for object 'sys software update': '     auto-check enabled'
2024-07-23T17:46:13Z - WARNING - tmconfpy.parser - UNRECOGNIZED LINE for object 'sys software update': '     auto-phonehome enabled'
2024-07-23T17:46:13Z - WARNING - tmconfpy.parser - UNRECOGNIZED LINE for object 'fatal-grace-time': '	time 500'
2024-07-23T17:46:13Z - WARNING - tmconfpy.parser - UNRECOGNIZED LINE for object 'fatal-grace-time': '	enabled yes'
2024-07-23T17:46:13Z - WARNING - tmconfpy.parser - UNRECOGNIZED LINE for object 'sys software update': '     auto-phonehome enabled'
2024-07-23T17:46:13Z - WARNING - tmconfpy.parser - UNRECOGNIZED LINE for object 'fatal-grace-time': '	time 500'
2024-07-23T17:46:13Z - WARNING - tmconfpy.parser - UNRECOGNIZED LINE for object 'fatal-grace-time': '	enabled yes'


#### tmconf in object format

In [4]:
tmconf.dict

{'apm application-family /Common/Audio': {'id': '16588'},
 'apm application-family /Common/Audio_Video': {'id': '16590'},
 'apm application-family /Common/Instant_Messaging': {'id': '16476'},
 'apm application-family /Common/Music': {'id': '16574'},
 'apm application-family /Common/News_and_Media': {'id': '16538'},
 'apm application-family /Common/Unknown': {'id': '16579'},
 'apm application-family /Common/Video': {'id': '16589'},
 'apm application-family /Common/Web_Based_Email': {'id': '16573'},
 'apm application /Common/archive_audio': {'app-family': '/Common/Audio',
  'app-family-id': '16588',
  'description': '"Internet Archive is an online digital library that holds digitized materials such as website histories, music, movies, images and digital books. This classification will match Archive\'s audio content."',
  'id': '4018'},
 'apm application /Common/http_file_audio': {'app-family': '/Common/Audio',
  'app-family-id': '16588',
  'description': '"This classification will match 

#### tmconf in tabular format

Each entry in tmconf is a row. Each row has three columns, 1) path, 2) name and 3) object.

In [5]:
tmconf.tabular

[tabularTmconf(path='apm application-family', name='/Common/Audio', object={'id': '16588'}),
 tabularTmconf(path='apm application-family', name='/Common/Audio_Video', object={'id': '16590'}),
 tabularTmconf(path='apm application-family', name='/Common/Instant_Messaging', object={'id': '16476'}),
 tabularTmconf(path='apm application-family', name='/Common/Music', object={'id': '16574'}),
 tabularTmconf(path='apm application-family', name='/Common/News_and_Media', object={'id': '16538'}),
 tabularTmconf(path='apm application-family', name='/Common/Unknown', object={'id': '16579'}),
 tabularTmconf(path='apm application-family', name='/Common/Video', object={'id': '16589'}),
 tabularTmconf(path='apm application-family', name='/Common/Web_Based_Email', object={'id': '16573'}),
 tabularTmconf(path='apm application', name='/Common/archive_audio', object={'app-family': '/Common/Audio', 'app-family-id': '16588', 'description': '"Internet Archive is an online digital library that holds digitized

In [6]:
# within python the tabular format uses named tuples. They allow accessing the columns by name.

row_no = 1

display(
    tmconf.tabular[row_no].path,
    tmconf.tabular[row_no].name,
    tmconf.tabular[row_no].object
)

'apm application-family'

'/Common/Audio_Video'

{'id': '16590'}

#### Display defined ciphers for each client-ssl profile

In [7]:
[(entry.name, entry.object.get('ciphers')) for entry in tmconf.tabular if entry.path == 'ltm profile client-ssl']

[('clientssl', 'DEFAULT'),
 ('clientssl-insecure-compatible', 'ALL:!DH:!ADH:!EDH:@SPEED'),
 ('wom-default-clientssl', None),
 ('crypto-server-default-clientssl', 'DHE-RSA-AES256-GCM-SHA384'),
 ('clientssl-secure', 'ecdhe:rsa:!sslv3:!rc4:!exp:!des'),
 ('splitsession-default-clientssl', None),
 ('clientssl-quic', 'none')]

#### Search for an object named 'aol' and return the path and object data

In [8]:
display([(entry.path, entry.object) for entry in tmconf.tabular if entry.name == 'aol'])

[('ltm data-group internal',
  {'records': {'64.12.96.0/19': {},
    '195.93.16.0/20': {},
    '195.93.48.0/22': {},
    '195.93.64.0/19': {},
    '195.93.96.0/19': {},
    '198.81.0.0/22': {},
    '198.81.8.0/23': {},
    '198.81.16.0/20': {},
    '202.67.65.128/25': {},
    '205.188.112.0/20': {},
    '205.188.146.144/30': {},
    '205.188.192.0/20': {},
    '205.188.208.0/23': {},
    '207.200.112.0/21': {}},
   'type': 'ip'})]

#### Find duplicate object names and their paths

In [9]:
from collections import defaultdict

# use defaultdict to create a dictionary with a list as the default value
name_to_path_mapping = defaultdict(list)

# iterate over the tabular data and append the path to the list of paths for each name
{name_to_path_mapping[name].append(path) for path, name, _ in tmconf.tabular}

# filter the dictionary to only include names with more than one path
duplicate_object_names = [(name, paths) for name, paths in name_to_path_mapping.items() if len(paths) > 1]

display(duplicate_object_names)

[('/Common/f5-default', ['ltm cipher rule', 'ltm cipher group']),
 ('/Common/f5-ecc', ['ltm cipher rule', 'ltm cipher group']),
 ('/Common/f5-hw_keys', ['ltm cipher rule', 'ltm cipher group']),
 ('/Common/f5-secure', ['ltm cipher rule', 'ltm cipher group']),
 ('/Common/f5-quic', ['ltm cipher rule', 'ltm cipher group']),
 ('/Common/f5-fips', ['ltm cipher rule', 'ltm cipher group']),
 ('/Common/f5-cc-stip', ['ltm cipher rule', 'ltm cipher group'])]

#### Find exactly where `/Common/default.key` is used in objects

In [ ]:
# install and use the awesome boltons package
!pip install boltons
from boltons.iterutils import research

In [11]:
# Use a list comprehension to create a list of dicts.
# The boltons research function to search for /Common/default.key either in keys or values

search_string = '/Common/default.key'

results = [
    # create a dict with path, object_name, and location_within_object keys
    {'path': path, 'object_name': name, 'path_within_object': used_at_position[0]}
    # enumerate over tabular data
    for path, name, obj in tmconf.tabular
    # filter out non-dict objects
    if isinstance(obj, dict) 
    # use the boltons research function to search for the search_string in keys or values and store the exact position
    and (used_at_position := research(obj, query=lambda path, key, value: key == search_string or value == search_string))
]

display(results)

[{'path': 'ltm profile client-ssl',
  'object_name': 'clientssl',
  'path_within_object': (('cert-key-chain', 'default', 'key'),
   '/Common/default.key')},
 {'path': 'ltm profile client-ssl',
  'object_name': 'clientssl-insecure-compatible',
  'path_within_object': (('cert-key-chain', 'default', 'key'),
   '/Common/default.key')},
 {'path': 'ltm profile client-ssl',
  'object_name': 'wom-default-clientssl',
  'path_within_object': (('cert-key-chain', 'default', 'key'),
   '/Common/default.key')},
 {'path': 'ltm profile client-ssl',
  'object_name': 'crypto-server-default-clientssl',
  'path_within_object': (('cert-key-chain', 'default', 'key'),
   '/Common/default.key')},
 {'path': 'ltm profile client-ssl',
  'object_name': 'clientssl-secure',
  'path_within_object': (('cert-key-chain', 'default', 'key'),
   '/Common/default.key')},
 {'path': 'ltm profile client-ssl',
  'object_name': 'splitsession-default-clientssl',
  'path_within_object': (('cert-key-chain', 'default', 'key'),
   '

In [12]:
# pragmatic command line approach
# use the tmconfpy CLI to search for /Common/default.key and display the results as JSONL redirecting stderr to /dev/null
!( tmconfpy test.tmconf --format jsonl | grep '/Common/default.key' ) 2>/dev/null

{"path": "ltm profile client-ssl", "name": "clientssl", "object": {"alert-timeout": "indefinite", "allow-dynamic-record-sizing": "disabled", "authenticate": "once", "authenticate-depth": "9", "ca-file": "none", "cache-size": "262144", "cache-timeout": "3600", "cert-extension-includes": ["basic-constraints", "subject-alternative-name"], "cert-key-chain": {"default": {"cert": "/Common/default.crt", "chain": "none", "key": "/Common/default.key", "passphrase": "none"}}, "cert": "/Common/default.crt", "chain": "none", "ciphers": "DEFAULT", "client-cert-ca": "none", "crl-file": "none", "handshake-timeout": "10", "key": "/Common/default.key", "maximum-record-size": "16384", "mod-ssl-methods": "disabled", "mode": "enabled", "options": ["dont-insert-empty-fragments", "no-tlsv1.3", "no-dtlsv1.2"], "passphrase": "none", "peer-cert-mode": "ignore", "renegotiate-max-record-delay": "indefinite", "renegotiate-period": "indefinite", "renegotiate-size": "indefinite", "renegotiation": "enabled", "secure